In [1]:
from envs.cartpole import Cartpole
from controllers.constant import target
from controllers.QP import QP_CBF_controller
from filters.disc import disc
from inv_set.cartpole import trap
from filters.NN_filters import FCN
from inv_set.inv_ped import cbf
import numpy as np
import matplotlib.pyplot as plt
import torch as th

# initialize cartpole
inv_set = trap(0.5, -5)
disc_steps = 2
env = Cartpole(inv_set, disc_steps)

# initialize network
input_size = 4
output_size = 2
n_layers = 5
size=1000
activation = 'relu'
lr = 5e-4
sc = 1
gamma_pos = 1
gamma_neg = 5
model = FCN(input_size, output_size, n_layers, size, activation, lr, sc, gamma_pos, gamma_neg)
model.FCN.train()

# train network
num_states = 10000
num_inputs = 300
epochs = 200
num_log = 4

train_losses = []
val_losses = []
best_loss = float('inf')
print("Sample Data 1/2")
val_states, val_inputs, val_labels = env.sample_data(num_states, num_inputs)
print("Sample Data 2/2")
train_states, train_inputs, train_labels = env.sample_data(num_states, num_inputs)
plt.scatter(val_states[:, 0], val_states[:, 1])
plt.savefig('exps/cartpole/states.png')
plt.clf()
print("Training model...")
for epoch in range(epochs):
    for i in range(5):
        train_loss = model.update2(train_states, train_inputs, train_labels, env)
    model.scheduler.step()
    if epoch % 1 == 0:
        train_losses.append(train_loss)
        val_loss = model.get_val_loss(val_states, val_inputs, val_labels, env)
        val_losses.append(val_loss)
        if val_loss < best_loss:
            th.save({
            'model_state_dict': model.FCN.state_dict(),
            'optimizer_state_dict': model.optimizer.state_dict(),
            }, 'exps/cartpole/model.pth')
        train_losses = train_losses[-500:]
        val_losses = val_losses[-500:]
        print("epoch:", epoch, 'train loss:', train_loss)
        print("epoch:", epoch, 'val loss:', val_loss)

model.FCN.eval()

/home/artur/miniconda3/envs/fsrl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/artur/miniconda3/envs/fsrl/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


Sample Data 1/2
Sampled states
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled inputs
Starting next states calculation
Sample Data 2/2
Sampled states
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled States again
Sampled inputs
Starting next states calculation
Training model...
epoch: 0 train loss: 0.014

Sequential(
  (0): Linear(in_features=4, out_features=1000, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1000, out_features=1000, bias=True)
  (3): ReLU()
  (4): Linear(in_features=1000, out_features=1000, bias=True)
  (5): ReLU()
  (6): Linear(in_features=1000, out_features=1000, bias=True)
  (7): ReLU()
  (8): Linear(in_features=1000, out_features=1000, bias=True)
  (9): ReLU()
  (10): Linear(in_features=1000, out_features=2, bias=True)
  (11): Identity()
)

<Figure size 640x480 with 0 Axes>

In [4]:
%load_ext autoreload

# Set autoreload to update modules automatically before executing code
%autoreload 2
from envs.cartpole import Cartpole
from controllers.constant import target
from controllers.QP import QP_CBF_controller
from filters.disc import disc
from inv_set.cartpole import trap
from filters.NN_filters import FCN
from inv_set.inv_ped import cbf
import numpy as np
import matplotlib.pyplot as plt
import torch 
inv_set = trap(0.5, -5)
disc_steps = 2
env = Cartpole(inv_set, disc_steps)
input_size = 4
output_size = 2
n_layers = 5
size=1000
activation = 'relu'
lr = 5e-4
sc = 1
gamma_pos = 1
gamma_neg = 5
nn_filter = FCN(input_size, output_size, n_layers, size, activation, lr, sc, gamma_pos, gamma_neg)
checkpoint = torch.load('exps/cartpole/model.pth', map_location=torch.device('cpu'))
nn_filter.FCN.load_state_dict(checkpoint['model_state_dict'])
nn_filter.FCN.eval()

class bang():
    def __init__(self):
        pass

    def forward(x, t):
        if t <= 10:
            return 1
        elif t <= 20:
            return -1
        else:
            return 1
    
  
target_controller = bang()

disc_filter = disc(env, 1000)
disc_controller = QP_CBF_controller(bang, disc_filter, 1)

nn_controller = QP_CBF_controller(bang, nn_filter, 1)

# run system and log results
print("Running exps...")
state = env.reset()
t = 300
traj_disc = env.run_system(state, t, disc_controller)
traj_nn = env.run_system(state, t, nn_controller)
plt.scatter(state[0], state[1], c='r')

plt.xlim(-1, 1)
plt.ylim(-6, 6)
plt.plot(traj_disc[:, 0], traj_disc[:, 1], label='disc')
plt.plot(traj_nn[:, 0], traj_nn[:, 1], label='nn')
plt.legend()
plt.savefig('exps/cartpole/traj.png')
plt.clf()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Running exps...
Starting next states calculation
Starting next states calculation
Starting next states calculation
Starting next states calculation


/home/artur/miniconda3/envs/fsrl/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/artur/miniconda3/envs/fsrl/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/artur/miniconda3/envs/fsrl/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/artur/miniconda3/envs/fsrl/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/artur/miniconda3/envs/fsrl/lib/python3.8/site-packages/sklearn/svm/_

Starting next states calculation
Starting next states calculation
Starting next states calculation


<Figure size 640x480 with 0 Axes>

In [9]:
from envs.cartpole import Cartpole
from controllers.constant import target
from controllers.QP import QP_CBF_controller
from filters.disc import disc
from inv_set.cartpole import trap
from filters.NN_filters import FCN
from inv_set.inv_ped import cbf
import numpy as np
import matplotlib.pyplot as plt
import torch 
inv_set = trap(0.5, -5)
disc_steps = 2
env = Cartpole(inv_set, disc_steps)
input_size = 4
output_size = 2
n_layers = 5
size=1000
activation = 'relu'
lr = 5e-4
sc = 1
gamma_pos = 1
gamma_neg = 5
nn_filter = FCN(input_size, output_size, n_layers, size, activation, lr, sc, gamma_pos, gamma_neg)
checkpoint = torch.load('exps/cartpole/model.pth', map_location=torch.device('cpu'))
nn_filter.FCN.load_state_dict(checkpoint['model_state_dict'])
model.FCN.eval()

model.forward([0,0,0,0])

/home/artur/miniconda3/envs/fsrl/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


(tensor([[0.0101]], grad_fn=<SliceBackward0>),
 tensor([-0.0027], grad_fn=<SelectBackward0>))

In [7]:
np.savez("exps/cartpole/training_data.npz", train_states = train_states, train_inputs = train_inputs, train_labels = train_labels)

In [8]:
np.savez("exps/cartpole/validation_data.npz", val_states = val_states, val_inputs = val_inputs, val_labels = val_labels)